In [17]:
# PyTorch Implementation
import torch
import torch.nn as nn
import torch.nn.functional as F

class Mlp_pytorch(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

in_features = 20
hidden_features = 50
out_features = 10

model = Mlp_pytorch(in_features, hidden_features, out_features)

n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {n_parameters}")

Number of parameters: 1560


In [18]:
torch.save(model.state_dict(), 'mlp.pth')

In [19]:
# TensorFlow Implementation
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import layers, models

class Mlp_tensorflow(tf.keras.layers.Layer):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer="gelu", drop=0.):
        super().__init__()
        self.in_features = in_features
        self.hidden_features = hidden_features or in_features
        self.out_features = out_features or in_features
        self.act_layer = Activation(act_layer)
        self.drop = Dropout(drop)
        self.fc1 = Dense(self.hidden_features, name="fc1")
        self.fc2 = Dense(self.out_features, name="fc2")

    def build(self, input_shape):
        self.fc1.build((input_shape[0], self.in_features))
        self.fc2.build((input_shape[0], self.hidden_features))
        super().build(input_shape)

    def call(self, x):
        x = self.fc1(x)
        x = self.act_layer(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

# Define the input shape
input_shape = (None, 20)  # Batch size, Example input shape

# Instantiate the TensorFlow model as part of a Model
inputs = tf.keras.Input(shape=(20,))
tf_mlp_layer = Mlp_tensorflow(in_features=20, hidden_features=50, out_features=10)
outputs = tf_mlp_layer(inputs)
tf_model = models.Model(inputs, outputs)

# Build the model to create the layer's weights
tf_model.build(input_shape)
n_parameters = tf_model.count_params()
print(f"Number of parameters: {n_parameters}")

Number of parameters: 1560


In [20]:
# Load PyTorch model and transfer weights to TensorFlow model
pytorch_model = Mlp_pytorch(in_features, hidden_features, out_features)
pytorch_model.load_state_dict(torch.load('mlp.pth', map_location=torch.device('cpu')))
pytorch_model.eval()

Mlp_pytorch(
  (fc1): Linear(in_features=20, out_features=50, bias=True)
  (act): GELU(approximate='none')
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (drop): Dropout(p=0.0, inplace=False)
)

In [21]:
# Helper function to convert PyTorch tensors to NumPy arrays
def pt_to_np(tensor):
    return tensor.detach().cpu().numpy()

In [22]:
# Map PyTorch weights to TensorFlow model
tf_mlp_layer.fc1.set_weights([pt_to_np(pytorch_model.fc1.weight.T), pt_to_np(pytorch_model.fc1.bias)])
tf_mlp_layer.fc2.set_weights([pt_to_np(pytorch_model.fc2.weight.T), pt_to_np(pytorch_model.fc2.bias)])

In [23]:
# Save TensorFlow model weights
tf_model.save_weights('mlp_tf.weights.h5')

In [24]:
# Load TensorFlow model weights and verify parameters
load_tf_model = models.Model(inputs, tf_mlp_layer(inputs))
load_tf_model.build(input_shape)
load_tf_model.load_weights('mlp_tf.weights.h5')
n_parameters = load_tf_model.count_params()
print(f"Number of parameters: {n_parameters}")

Number of parameters: 1560
